In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
from pandas_datareader import data as pdr

yf.pdr_override()
stock = input("Enter a ticker symbol: ")
start = str(input("Enter Start Date: "))
end = str(input("Enter End Date: "))
df = pdr.get_data_yahoo(stock, start, end)
df.to_csv('data.csv')

df = pd.read_csv('data.csv', index_col='Date', parse_dates=True)

emasUsed=[3,5,8,10,12,15,30,35,40,45,50,60]
for x in emasUsed:
    ema=x
    df["Ema_"+str(ema)]=round(df.iloc[:,4].ewm(span=ema, adjust=False).mean(),2)
print(df.tail())

pos=0
num=0
percentchange=[]

for i in df.index:
    cmin=min(df["Ema_3"][i],df["Ema_5"][i],df["Ema_8"][i],df["Ema_10"][i],df["Ema_12"][i],df["Ema_15"][i])
    cmax=max(df["Ema_30"][i],df["Ema_35"][i],df["Ema_40"][i],df["Ema_45"][i],df["Ema_50"][i],df["Ema_60"][i])

    close=df["Adj Close"][i]

    if(cmin>cmax):
        if(pos==0):
            bp=cmin    
            pos=1       
            print("Bought at "+str(bp))

    elif(cmin<cmax):
        if(pos==1):
            pos=0
            sp=cmax    
            print("Sold at "+str(bp))
            pc=(sp/bp-1)*100
            percentchange.append(pc)
    
    if(num==df["Adj Close"].count()-1 and pos ==1):
        pos=0
        sp=cmax   
        print("Sold at "+str(bp))
        pc=(sp/bp-1)*100
        percentchange.append(pc)
    num+=1

print(percentchange)

gains=0
ng=0
losses=0
nl=0
totalR=1

for i in percentchange:
    if(i>0):
        gains+=1
        ng+=1
    else:
        losses+=i
        nl+=1
    totalR=totalR*((i/100)+1)

totalR=round((totalR-1)*100,2)

if(ng>0):
    avgGain=gains/ng
    maxR=str(max(percentchange))
else:
    avgGain=0
    maxR="undefined"

if(nl>0):
    avgLoss=losses/nl
    maxL=str(max(percentchange))
    ratio=str(-avgGain/avgLoss)
else:
    avgLoss=0
    maxL="undefined"
    ratio="inf"

if(ng>0 or nl>0):
    battingAvg=ng/(ng+nl)
else:
    battingAvg=0

print()
print("Resluts for "+ stock +" going back to "+str(df.index[0])+", Sample size: "+str(ng+nl)+" trade")
print("EMAs used: "+str(emasUsed))
print("Batting Avg "+str(battingAvg))
print("Gain/loss ratio: "+ ratio)
print("Average Gain: "+str(avgGain))
print("Average Loss: "+str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ "trades: "+ str(totalR)+"%" )
print()

Enter a ticker symbol: BTC-USD
Enter Start Date: 2019-01-01
Enter End Date: 2022-01-01
[*********************100%***********************]  1 of 1 completed
                    Open          High           Low         Close  \
Date                                                                 
2021-12-28  50679.859375  50679.859375  47414.210938  47588.855469   
2021-12-29  47623.871094  48119.742188  46201.496094  46444.710938   
2021-12-30  46490.605469  47879.964844  46060.312500  47178.125000   
2021-12-31  47169.371094  48472.527344  45819.953125  46306.445312   
2022-01-01  46311.746094  47827.312500  46288.484375  47686.812500   

               Adj Close       Volume     Ema_3     Ema_5     Ema_8    Ema_10  \
Date                                                                            
2021-12-28  47588.855469  33430376883  49093.38  49411.64  49384.32  49329.93   
2021-12-29  46444.710938  30049226299  47769.05  48422.66  48731.07  48805.34   
2021-12-30  47178.125000  266